In [1]:
# ==== Librerías principales ====
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# ==== Comprobación de dispositivo ====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# ==== Semillas para reproducibilidad ====
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if device.type == "cuda":
    torch.cuda.manual_seed_all(SEED)

# Configuración de gráficos
plt.style.use("seaborn-v0_8")


Usando dispositivo: cuda
GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [ ]:
# Carga y comprobación inicial de datos para LSTM (Amazon_stock_data.csv)
import pandas as pd
import numpy as np
from datetime import datetime

# --- 1) Cargar CSV ---
path = "/mnt/data/Amazon_stock_data.csv"
df = pd.read_csv(path)

# --- 2) Normalizar nombres de columnas (evitar espacios/casos) ---
df.columns = [c.strip().replace(" ", "_") for c in df.columns]

# --- 3) Intentar detectar columna de fecha ---
date_col_candidates = [c for c in df.columns if c.lower() in ["date", "fecha", "timestamp", "datetime"]]
date_col = date_col_candidates[0] if date_col_candidates else None
if date_col:
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    # Orden cronológico y set index si aplica
    df = df.sort_values(by=date_col).reset_index(drop=True)
else:
    # Si no hay columna fecha, crear índice temporal ficticio
    date_col = "idx_time"
    df[date_col] = pd.RangeIndex(start=0, stop=len(df), step=1)

# --- 4) Tipos y valores faltantes ---
dtypes = df.dtypes.astype(str).to_frame("dtype")
na_counts = df.isna().sum().to_frame("na_count")
summary_dtypes_na = dtypes.join(na_counts)

# --- 5) Duplicados (por fila completa y por fecha si existe) ---
full_dupes = int(df.duplicated().sum())
date_dupes = int(df.duplicated(subset=[date_col]).sum()) if date_col in df.columns else 0

# --- 6) Estadísticas numéricas básicas ---
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
desc = df[numeric_cols].describe().T if numeric_cols else pd.DataFrame()

# --- 7) Mostrar dataframes al usuario ---
preview_rows = min(25, len(df))
print("Vista previa (primeras filas ordenadas por fecha):")
display(df.head(preview_rows))
print("Tipos y valores faltantes por columna:")
display(summary_dtypes_na)
if not desc.empty:
    print("Estadísticas descriptivas (columnas numéricas):")
    display(desc)

# --- 8) Informe textual breve ---
print("Archivo cargado:", path)
print("Filas x Columnas:", df.shape)
print("Columna de fecha detectada:", date_col)
print("Duplicados de fila completos:", full_dupes)
print("Duplicados por fecha:", date_dupes)
print("Columnas numéricas:", numeric_cols)
